In [1]:
import numpy as np
import jax
import jax.numpy as jnp


In [ ]:
def paulis(dtype=jnp.complex64):
    '''Creates single-qubit basis operators'''
    sx = jnp.array([[0., 1.] , [1., 0.]] , dtype=dtype )
    sy = jnp.array([[0., -1j], [1j, 0.]] , dtype=dtype )
    sz = jnp.array([[1., 0.] , [0., -1.]], dtype=dtype )
    id2 =jnp.array([[1., 0.] , [0., 1.]] , dtype=dtype )
    return sx, sy, sz, id2

def kron_n(ops):
    '''Tensor product of a list of operators'''
    out = ops[0]
    for A in ops[1:]:
        out = jnp.kron(out, A)
    return out

class Hamiltonian:

    '''
    Builds Hamiltonian with a potential barrier in a qubit. 
    We can chose the type (diffusion or transport)
    and the mode (spin picture or domain wall picture)
    '''

    def __init__(self, N, lmd, V, V_index, J, hamiltonian_type, mode):
        self.N = N
        self.lmd = lmd
        self.V = V
        self.V_index = V_index
        self.J = J
        self.H_type = hamiltonian_type
        self.mode = mode

        self.sx, self.sy, self.sz, self.id2 = paulis()

        self.sx_list, self.sy_list, self.sz_list = self._initialize_operators()
        self.couplings = self._calculate_couplings()

        if self.mode == 'sp':
            self.H = self._build_spin_hamiltonian()
        elif self.mode == 'dw':
            self.H = self._build_domain_wall_hamiltonian()

    
    def _initialize_operators(self):
        '''Setup operators for individual qubits
            for each value of i it puts the paulis in different positions of the list, 
            then does IxIxI...sigma_ixIxI...xI
        '''
        sx_list, sy_list, sz_list = [], [], []
        for i in range(self.N):
            #list of 2x2 identity matrices
            op_list = [self.id2] * self.N
            #replace i-th element with sigma_x
            op_list[i] = self.sx
            #create matrices of 2^Nx2^N
            sx_list.append(kron_n(op_list))
            #do the same for sigma_y and sigma_z
            op_list[i] = self.sy
            sy_list.append(kron_n(op_list))
            op_list[i] = self.sz
            sz_list.append(kron_n(op_list))

        return sx_list, sy_list, sz_list
    

    def _mirror_symmetric_terms(size, factor):
        strengths = jnp.zeros(size)
        for i in range(0,size):
            strengths[i] = -0.5*factor*np.sqrt((i+1)*(size-i))
        return strengths
    
    def _calculate_couplings(self):
        """
        Calculate couplings for lambda, J and z fields for different situations (standard or domain walls)
        If errors are zeros we avoid calling the errors.py script

        Returns: 
            couplings: Dictionary containing all necesary values for the hamiltonian, indexed by their name
                       with errors added if requested

        """
        #Define all Hamiltonian couplings as dictionary
        couplings = {}

        #Define ideal transverse fields
        if self.H_type == 'diffusion':
            error_free_l = [self.lmd]*(self.N)
        elif self.H_type == 'transport':
            error_free_l = error_free_l = self._mirror_symmetric_terms(self.n_spins, self.lmd)
        else:
            raise ValueError(f"{self.H_type} is not a valid Hamiltonian type. Enter 'diffusion' or 'transport'")

        couplings["lambda"] = error_free_l

        #errors in domain wall couplings
        if self.J:  #not used in standard encoding
            error_free_j = [self.J]*(self.n_spins) #two virtual spins
            couplings["J"] = 2*[self.J] + error_free_j

        return couplings
    

    def _build_spin_hamiltonian(self):

        H = jnp.zeros((2**self.N, 2**self.N), dtype=jnp.complex64)

        l_terms = self.couplings["lambda"]

        #Transverse field but not in first spin
        for i in range(0, self.n_spins-1):          #all spins with transverse field
            H += -l_terms[i] * self.sx_list[i]*self.sx_list[i+1]
            H += -l_terms[i] * self.sy_list[i]*self.sy_list[i+1]

        H += self.V*self.sz_list[self.V_index]
        
        return H

    

    def _build_domain_wall_hamiltonian(self):
        '''Create a different type of hamiltonian depending on the string passed'''

        H = jnp.zeros((2**self.N, 2**self.N), dtype=jnp.complex64)

        l_terms = self.couplings["lambda"]
        j_terms = self.couplings["J"]

        #Transverse field but not in first spin
        for i in range(0, self.n_spins):          #all spins with transverse field
            H += -l_terms[i] * self.sx_list[i]

        #Virtual qubit up at the start of chain
        H += -j_terms[0]*self.sz_list[0]

        #Virtual qubit down at end of chain
        H += j_terms[-1]*self.sz_list[self.n_spins-1]

        #Interaction terms with the rest of the spins except for the last one
        for i in range(0, self.n_spins-1):
            H += j_terms[i+1]*self.sz_list[i]*self.sz_list[i+1]

        #Potential barrier
        H += -0.5*self.V*self.sz_list[self.V_index]*self.sz_list[self.V_index +1]
        
        return H
    


    

In [8]:
N = 6
lmd = 0.02272
J = 1
V = 1*lmd
V_index = 3
hamiltonian_type = 'diffusion'
mode = 'spin'

H = Hamiltonian(N, lmd, V, V_index, J, hamiltonian_type, mode)